Import Problems:

* value of columns needs nice parsing
* 2 header lines?
* What about time ref values in columns and in rows
* Grouping if disctrict code happens to be multiple times in it

In [3]:
import requests
import pandas as pd
import json
import os
import IPython
import random
import csv
import re
import chardet
import reboting
#injected variables
#url='http://www.wien.gv.at/statistik/ogd/vie_403.csv'
url='http://www.wien.gv.at/politik/wahlen/ogd/bv051_99999999_9999_spr.csv'
#url='http://service.stmk.gv.at/ogd/OGD_Data_ABT17/statistik/BEV_GESCHL/STMK_01012016_SEX.csv'
name='testing'
description='desc'
publisher='bla'
portal="https"
#data reading and preparation
filename = reboting.readandcleancsv( url )
#print(filename)
df = pd.read_csv(filename,sep=';', thousands='.', decimal=',')
#test if SUB_DISTRICT_CODE exists
districtCode =""
if 'DISTRICT_CODE' in df.columns:
    districtCode="DISTRICT_CODE"
if 'SUB_DISTRICT_CODE' in df.columns:
    districtCode='SUB_DISTRICT_CODE'
if 'LAU_CODE' in df.columns:
    districtCode='LAU_CODE'
df = df.drop_duplicates(subset=districtCode)
df.columns=df.columns.str.replace('#','')
df.columns=df.columns.str.replace('.','')
df.columns=df.columns.str.replace(':','')
df.columns=df.columns.str.replace('"','')
df.columns=df.columns.str.replace(' ','')
numeric_columnlist = list(df._get_numeric_data().columns)
#filter out empty column names
string_columns=[item for item in list(df.columns) if item not in numeric_columnlist and item!='']
numeric_columnlist=[item for item in numeric_columnlist if item!='' and item!='DISTRICT_CODE' and item!='SUB_DISTRICT_CODE' and item!='LAU_CODE']
#last numeric field as entity field for know.
valueField = random.choice(numeric_columnlist)
entityField = string_columns[-1]
data_dict = df.to_dict(orient='records')
#delete temp file
os.remove(filename)
requestOBJ = {
        "data" : data_dict,
        "meta": {
            "name": name + " "+valueField,
            "sourceOrganization": publisher,
            "source": publisher,
            "description": description,
            "publisher_name": publisher,
            "publisher_homepage": portal,
            "publisher_contact": "EMAIL ADRESS OF PUBLISHER",
            "publisher_tags": ["City of Vienna", "Demographie", "Population"],
            "accessUrl": "/api/data/",
            "accessFormat": "json",
            "frequency": "INSERT EXPECTED DATA UPDATE CYCLE",
            "license": "Creative Commons Namensnennung 3.0 Österreich",
            "citation": "INSERT CITATION IF AVAILABLE",
            "isoField": districtCode,
            "entityField": entityField,
            "timeField": "null",
            "timeDimension": "false",
            "timeUnit": "year",
            "valueField": valueField,
            "unit": "Value:",
            "colors": ["#ffc971", "#ffb627", "#ff9505", "#e2711d", "#cc5803"],
            "legendtitles": ["low", "med-low", "med", "med-high", "high"],
            "tooltip": [{"label": valueField, "field": "data:"+valueField }],
            "theme": "light"
        },
        "parameters": {
            "source": "manual",
            "provider": "reboting",
            "layer": "choropleth"
        }
    }
#print(json.dumps(requestOBJ, indent=3, sort_keys=True))
r = requests.post("http://52.166.116.205:2301/manual", json=requestOBJ)
url = 'https://doh.23degrees.io/viz/'+r.json()['slug']
iframe= '<div class="intrinsic-container" style="position: relative; padding-bottom: 56.25%;height: 0; overflow: hidden; width: 100%;height: auto;"><iframe src="' + url + '" allowfullscreen style=" position: absolute; top: 0; left: 0; width: 100%; height: 100%; "></iframe></div>'
#iframe= '<div class="intrinsic-container" style="position: relative; padding-bottom: 56.25%;height: 0; overflow: hidden; width: 100%;height: auto;"><iframe src="' + url + '" allowfullscreen style=" position: absolute; top: 0; left: 0; width: 100%; height: 100%; "></iframe></div>'
#print(url)
IPython.display.HTML(iframe)